# Data Wrangling

## Data set

1. Box Office Mojo
2. The Movie DB
3. The Numbers
4. IMDB

## Import relevant libraries:

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import sqlite3
import seaborn as sns
%matplotlib inline

## 1. Box Office Mojo dataset

Open the bom_movie

In [ ]:
# Open bom_movie dataframe:
bom_movie = pd.read_csv('zippedData/bom.movie_gross.csv')
bom_movie.head()

In [ ]:
#Create a new column from mapping title & year:
bom_movie['title_year'] = bom_movie.title + bom_movie['year'].astype(str)
bom_movie.head()

## 2. The Movie DB dataset

In [ ]:
# Open tmdb_movie dataframe:
tmdb_movie = pd.read_csv('zippedData/tmdb.movies.csv')
tmdb_movie.head()

In [ ]:
tmdb_movie.describe()

In [ ]:
#Get 'release_month' & 'release_year' from column 'release_date':
tmdb_movie['release_month'] = tmdb_movie['release_date'].map(lambda x: pd.to_datetime(x).month)
tmdb_movie['release_year'] = tmdb_movie['release_date'].map(lambda x: x[:4])
tmdb_movie.head()

In [ ]:
# Create a new column from mapping title & year:
tmdb_movie['title_year'] = tmdb_movie['title'] + tmdb_movie['release_year'].astype(str)
tmdb_movie.head()

In [ ]:
tmdb_movie.info()

## 3. The Numbers dataset

In [ ]:
#Open tn_movie dataframe:
tn_movie = pd.read_csv('zippedData/tn.movie_budgets.csv')
tn_movie.head()

In [ ]:
# Get 'release_month' & 'release_year' from release_date and create new columns:
tn_movie['release_month'] = tn_movie['release_date'].map(lambda x: pd.to_datetime(x).month)
tn_movie['release_year'] = tn_movie['release_date'].map(lambda x: x[-4:])
tn_movie.head()

In [ ]:
# Create a new column from mapping 2 columns 'movie' & 'release_year':
tn_movie['movie_year'] = tn_movie['movie'] + tn_movie['release_year'].astype(str)

In [ ]:
tn_movie.head()

In [ ]:
tn_movie.shape

In [ ]:
# Turn production_budget value in numeric values:
tn_movie["production_budget"] = [str(i).replace("$", "") for i in tn_movie["production_budget"]]
tn_movie["production_budget"] = [float(str(i).replace(",", "")) for i in tn_movie["production_budget"]]

In [ ]:
# Turn domestic_gross value in numeric values:
tn_movie["domestic_gross"] = [str(i).replace("$", "") for i in tn_movie["domestic_gross"]]
tn_movie["domestic_gross"] = [float(str(i).replace(",", "")) for i in tn_movie["domestic_gross"]]


In [ ]:
# Turn worldwide_gross value in numeric values:
tn_movie["worldwide_gross"] = [str(i).replace("$", "") for i in tn_movie["worldwide_gross"]]
tn_movie["worldwide_gross"] = [float(str(i).replace(",", "")) for i in tn_movie["worldwide_gross"]]

In [ ]:
#Create 7 new columns of gross revenue, profit & ROI:
tn_movie['international_gross'] = tn_movie.worldwide_gross - tn_movie.domestic_gross
tn_movie['domestic_profit'] = tn_movie.domestic_gross - tn_movie.production_budget
tn_movie['international_profit'] = tn_movie.international_gross - tn_movie.production_budget
tn_movie['worldwide_profit'] = tn_movie.worldwide_gross - tn_movie.production_budget
tn_movie['domestic_ROI'] = (tn_movie.domestic_gross - tn_movie.production_budget)/tn_movie.production_budget
tn_movie['international_ROI'] = (tn_movie.international_gross - tn_movie.production_budget)/tn_movie.production_budget
tn_movie['worldwide_ROI'] = (tn_movie.worldwide_gross - tn_movie.production_budget)/tn_movie.production_budget

In [ ]:
tn_movie.head()

In [ ]:
tn_movie.info()

## 4. IMDB dataset

In [ ]:
con = sqlite3.connect('zippedData/im.db')
cursor_obj = con.cursor()

In [ ]:
%%bash 

sqlite3 zippedData/im.db
.tables

In [ ]:
#Open movie_basics dataframe:
movie_basics_df = pd.read_sql("""
SELECT *
FROM movie_basics
""", con)
movie_basics_df.head()

### 4.1. Open directors dataframe:

In [ ]:
# Create a new column from mapping 'primary_title' & 'year'
movie_basics_df['title_year'] = movie_basics_df['primary_title'] + movie_basics_df['start_year'].astype(str)
movie_basics_df.head()

## Merging data

Merge 'tn_movie' dataframe with 'movie_basics' basing on 'movie_year' & 'title_year' 
to link genres with financial metrics:

In [ ]:
movie_profit_genre = pd.merge(tn_movie, movie_basics_df, left_on = "movie_year",right_on ="title_year", how = "inner" )
movie_profit_genre.head()

In [ ]:
movie_profit_genre.info()

In [ ]:
#Copy "genres" column and create a new column called "each_genre"
movie_profit_genre.loc[:, 'each_genre'] = movie_profit_genre['genres'].copy()
movie_profit_genre.head()

Sort string in "genres" columns in anphabet order:

In [ ]:

movie_profit_genre.loc[:, 'genres'] = movie_profit_genre['genres'].apply(lambda x: ', '.join(sorted(x.split(', '))) if x is not None else '')

movie_profit_genre.info()

In [ ]:
movie_profit_genre.columns

Split "genres" into single genre, from 1 row into multiple rows:

In [ ]:
movie_genres = movie_profit_genre.set_index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross', 'release_month', 'release_year', 'movie_year',
       'international_gross', 'domestic_profit', 'international_profit',
       'worldwide_profit', 'domestic_ROI', 'international_ROI',
       'worldwide_ROI', 'movie_id', 'primary_title', 'original_title',
       'start_year', 'runtime_minutes', 'genres', 'title_year']).apply(lambda x: x.str.split(',').explode()).reset_index()
movie_genres.head()

Calculate mean of all financial metrics of each genre

In [ ]:
mean_profit_genre = movie_genres.groupby('each_genre').mean(numeric_only = True).sort_values('worldwide_profit',ascending = False)
mean_profit_genre.style.format({'runtime_minutes': '{:,.2f}','production_budget': '{:,.2f}','domestic_gross': '{:,.2f}', 'worldwide_gross': '{:,.2f}', 'international_gross': '{:,.2f}',
       'domestic_profit': '{:,.2f}','international_profit': '{:,.2f}','worldwide_profit': '{:,.2f}','domestic_ROI': '{:.2f}','international_ROI': '{:.2f}', 'worldwide_ROI': '{:.2f}'})
 

Top 4 genre by worldwide profit:

In [ ]:
top4_genre_profit = mean_profit_genre.head(4)
top4_genre_profit.style.format({'runtime_minutes': '{:,.2f}','production_budget': '{:,.2f}','domestic_gross': '{:,.2f}', 'worldwide_gross': '{:,.2f}', 'international_gross': '{:,.2f}',
       'domestic_profit': '{:,.2f}','international_profit': '{:,.2f}','worldwide_profit': '{:,.2f}','domestic_ROI': '{:.2f}','international_ROI': '{:.2f}', 'worldwide_ROI': '{:.2f}'})

In [ ]:
#List of top 4 genre list by worldwide profit:
top4_genre_list = list(top4_genre_profit.index)
top4_genre_list

Filter dataframe including top 4 genres

In [ ]:
top4_genre_df = movie_genres.loc[movie_genres['each_genre'].isin(top4_genre_list)]
top4_genre_df.head().style.format({'runtime_minutes': '{:,.2f}','production_budget': '{:,.2f}','domestic_gross': '{:,.2f}', 'worldwide_gross': '{:,.2f}', 'international_gross': '{:,.2f}',
       'domestic_profit': '{:,.2f}','international_profit': '{:,.2f}','worldwide_profit': '{:,.2f}','domestic_ROI': '{:.2f}','international_ROI': '{:.2f}', 'worldwide_ROI': '{:.2f}'})
 

Indentify the mean, min, max of each financial metrics in top 4 genres:

In [ ]:
top4_genre_df.describe().style.format({'runtime_minutes': '{:,.2f}','production_budget': '{:,.2f}','domestic_gross': '{:,.2f}', 'worldwide_gross': '{:,.2f}', 'international_gross': '{:,.2f}',
       'domestic_profit': '{:,.2f}','international_profit': '{:,.2f}','worldwide_profit': '{:,.2f}','domestic_ROI': '{:.2f}','international_ROI': '{:.2f}', 'worldwide_ROI': '{:.2f}'})
 

In [ ]:
#Create intervals of production budget:
budget_interval = [0,20000000, 50000000, 100000000,200000000,300000000,400000000, float('inf')]
#Create a new column of production_budget intervals:
top4_genre_df.loc[:, 'budget_group']  = pd.cut(top4_genre_df.loc[:,'production_budget'], bins=budget_interval, labels=['<20M', '20M-50M', '50M-100M','100M-200M', '200M-300M','300M-400M','>400M'])

In [ ]:
top4_genre_df.head()

Use violin plot to find the correlation between worldwide ROI & production budget group:

#### Merge table to get Popularity & financial metrics & genres

Merge 'tn_movie' & 'tmdb_movie' dataframes to get popularity and ROI

In [ ]:
popularity_ROI = pd.merge(tn_movie,tmdb_movie, left_on = "movie_year", right_on = 'title_year', how = 'inner')
popularity_ROI.info()

Merge 'popularity_ROI' & 'movie_basics_df' dataframes to get genres information

In [ ]:
popularity_ROI_genres = pd.merge(popularity_ROI,movie_basics_df, left_on = "movie_year",right_on = 'title_year', how = 'inner')

In [ ]:
popularity_ROI_genres.info()

In [ ]:
#Sort string in "genres" columns in anphabet order:
popularity_ROI_genres.loc[:, 'genres'] = popularity_ROI_genres['genres'].apply(lambda x: ', '.join(sorted(x.split(', '))) if x is not None else '')
popularity_ROI_genres.head()

In [ ]:
#Copy "genres" column and create a new column called "each_genre"
popularity_ROI_genres.loc[:, 'each_genre'] = popularity_ROI_genres['genres'].copy()
popularity_ROI_genres.head()

In [ ]:
popularity_ROI_genres.describe().style.format({'runtime_minutes': '{:,.2f}','production_budget': '{:,.2f}','domestic_gross': '{:,.2f}', 'worldwide_gross': '{:,.2f}', 'international_gross': '{:,.2f}',
       'domestic_profit': '{:,.2f}','international_profit': '{:,.2f}','worldwide_profit': '{:,.2f}','domestic_ROI': '{:.2f}','international_ROI': '{:.2f}', 'worldwide_ROI': '{:.2f}'})

In [ ]:
#Split "each_genre" row into multiple rows:
popularity_ROI_each_genre = popularity_ROI_genres.set_index(['id_x', 'release_date_x', 'movie', 'production_budget',
       'domestic_gross', 'worldwide_gross', 'release_month_x',
       'release_year_x', 'movie_year', 'international_gross',
       'domestic_profit', 'international_profit', 'worldwide_profit',
       'domestic_ROI', 'international_ROI', 'worldwide_ROI', 'Unnamed: 0',
       'genre_ids', 'id_y', 'original_language', 'original_title_x',
       'popularity', 'release_date_y', 'title', 'vote_average', 'vote_count',
       'release_month_y', 'release_year_y', 'title_year_x', 'movie_id',
       'primary_title', 'original_title_y', 'start_year', 'runtime_minutes',
       'genres', 'title_year_y']).apply(lambda x: x.str.split(',').explode()).reset_index()
popularity_ROI_each_genre.head()

Filter 'popularity_ROI_each_genre' dataframe to get top 4 genres

In [ ]:
top4_genre_list

In [ ]:
top4_popularity_df = popularity_ROI_each_genre[popularity_ROI_each_genre['each_genre'].isin(top4_genre_list)]

In [ ]:
top4_popularity_df.head()

Create intervals of popularity:

In [ ]:

interval_popularity = [0,10, 20, 30, 40, float('inf')]
top4_popularity_df.loc[:, 'popularity_group']  = pd.cut(top4_popularity_df['popularity'], bins=interval_popularity, labels=['<10','10-20', '20-30', '30-40','>40'])

In [ ]:
top4_popularity_df.head()

# Conclusion:

1st dataframe: to get each_genre & financial metrics

In [ ]:
movie_genres.to_csv('zippedData/movie_genres_cleaned')

In [ ]:
pd.read_csv('zippedData/movie_genres_cleaned')

2nd dataframe: to get popularity, genres & financial metrics:

In [ ]:
popularity_ROI_each_genre.to_csv('zippedData/cleaned_popularity_profit_data')

In [ ]:
pd.read_csv('zippedData/cleaned_popularity_profit_data')